# MyMethods.jl -- Examples

This notebook illustrates basic functionalities of the MyMethods.jl package.

### Installation and Uninstallation of MyMethods.jl

In [2]:
# Install the package from GitHub (first time only) 
#using Pkg; 
#Pkg.add(url="https://github.com/thomaswiemann/MyMethods.jl");

# Uninstall 
#Pkg.rm("MyMethods")

   Updating git-repo `https://github.com/thomaswiemann/MyMethods.jl`


   Updating registry at `C:\Users\thomas\.julia\registries\General`


LoadError: package `MyMethods [18de7e97]` has same name or UUID as the active project

In [3]:
# Load MyMethods.jl into your Julia session 
using MyMethods

┌ Info: Precompiling MyMethods [18de7e97-ce2a-4ea9-8d3b-e4b0c3b0b515]
└ @ Base loading.jl:1278
┌ Warning: Package MyMethods does not have DataFrames in its dependencies:
│ - If you have MyMethods checked out for development and have
│   added DataFrames as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with MyMethods
└ Loading DataFrames into MyMethods from project dependency, future warnings for MyMethods are suppressed.


In [13]:
# Load additionally useful (but not necessary) packages
using Statistics # for a quantile function
using Plots; plotly() # for a plotting backend

┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots C:\Users\thomas\.julia\packages\Plots\D7Ica\src\backends.jl:373


Plots.PlotlyBackend()

### Exemplary Data  

Generate basic data to be used in the examples.

In [32]:
# Data parameters
N = 10000 # sample size
beta = [1,2,-1,4] # regression coefficients

# Data
_r = (1 .+ 1.5 .* (rand(N) .- 0.5)) # running variable
X = hcat(ones(N), rand(N,3)) # regressor matrix
y = X*beta.*_r + randn(N,1); # outcome simulation

### myLS
myLS() generates a (weighted) least squares object.

In [27]:
# Estimate least squares fit
fit_LS = myLS(y, X);

In [28]:
# Calculate and print standard errors
inference(fit_LS);

,coef,se,t-stat,p-val
,Float64,Float64,Float64,Float64
1,1.14651,0.0366492,31.2834,7.84219e-215
2,2.36292,0.0401474,58.8563,0.0
3,-1.17118,0.0402771,-29.0782,6.7794e-186
4,4.70413,0.0402201,116.96,0.0


### myLLR
myLLR generates a local linear regression object.

In [33]:
# Load the Statistics.jl package for the quantile function
using Statistics

In [47]:
# Estimate local constant regression with bandwidth h = 0.3
fit_LLR = myLLR(y, _r, K=0, h=0.4, 
    control = X[:, 2:end]);

In [48]:
# Calculate the local coefficients for quantiles of the running variable
_pred = quantile(_r, collect(1:100)/100);
coef_LLR = coef(fit_LLR, _pred);

In [50]:
# Plot coefficients as a function of the running variable
plot(_pred, coef_LLR,
title = "Local linear regression coefficients",
label = ["Const." "X1" "X2" "X3"])

<!DOCTYPE html>
 
 
 Plots.jl

# mybootstrap

In [51]:
# bootstrap myLS
res = mybootstrap(myLS, y, 100, X, data_args = [1], 
    get = coef, red = hcat∘vcat);

In [ ]:
# bootstrap myLLR
res = mybootstrap(myLLR, y, 10, ) # think about bootstrapping myLLR!